In [ ]:
inshp  = '/mnt/d/a3land2023/rule3_4/sample.shp'

dist = 500

angle_min = 0
angle_max = 45

angles_int = 45

outshp = '/mnt/d/a3land2023/rule3_4/cheese_buffer3.shp'
outpnt = '/mnt/d/a3land2023/rule3_4/polyvtx6.shp'

In [ ]:
import geopandas as gp
from shapely.geometry import Point
import numpy as np
from shapely import wkt
#from geopy.distance import azimuth

from glass.rd.shp import shp_to_obj
from glass.dtt.cg.obj import multipart_to_single
from glass.it.pd import df_to_geodf
from glass.prop.prj import get_epsg
from glass.wt.shp import df_to_shp
from glass.gobj import create_polygon

In [ ]:
epsg = get_epsg(inshp)

In [ ]:
pdf = shp_to_obj(inshp)

# Force geometries single part
pdf = multipart_to_single(pdf, 'Polygon')

# Get centroid
pdf["ctr"] = pdf.geometry.centroid

pdf['ctr_x'] = pdf.ctr.x
pdf['ctr_y'] = pdf.ctr.y

pdf.drop(['ctr'], axis=1, inplace=True)

In [ ]:
# Polygon vertices to a new Point GeoDataFrame
pnts = []

for i, row in pdf.iterrows():
    # Get vertices
    vtx = list(row.geometry.exterior.coords)
    
    # Get Point Geometries and attributes
    o = 1
    for v in vtx:
        d = {"geometry" : Point(v), "featid": i, "order": o}
        
        for attr in pdf.columns.values:
            if attr == "geometry": continue
            
            d[attr] = row[attr]
        
        pnts.append(d)
        
        o += 1

vtx_df = df_to_geodf(pnts, 'geometry', epsg)

#df_to_shp(vtx_df, outpnt)

In [ ]:
vtx_df["aty"] = vtx_df.geometry.y - vtx_df.ctr_y
vtx_df["atx"] = vtx_df.geometry.x - vtx_df.ctr_x

In [ ]:
vtx_df['theta'] = np.degrees(np.arctan2(vtx_df.atx, vtx_df.aty))

In [ ]:
vtx_df["azimute"] = np.where(
    vtx_df.theta < 0, vtx_df.theta + 360,
    vtx_df.theta
)
vtx_df["azimute"] = vtx_df.azimute.round(0).astype(int)

In [ ]:
# Get new coordinates for each vertice
vtx_df["new_x"] = vtx_df.geometry.x + (dist * np.sin(np.radians(vtx_df.azimute)))
vtx_df["new_y"] = vtx_df.geometry.y + (dist * np.cos(np.radians(vtx_df.azimute)))

In [ ]:
# Get Slices
slices = {}

minangle = 0
maxangle = angles_int

s = 1
while maxangle <= 360:
    slices[s] = (minangle, maxangle)
    
    minangle += angles_int
    maxangle += angles_int
    
    s+= 1

In [ ]:
slices

In [ ]:
# Classify points

vtx_df["slice_id"] = 0
vtx_df["slice_label"] = ''

for k in slices:
    vtx_df["slice_id"] = np.where(
        (vtx_df.azimute >= slices[k][0]) & (vtx_df.azimute < slices[k][1]),
        k, vtx_df.slice_id
    )
    
    vtx_df["slice_label"] = np.where(
        (vtx_df.azimute >= slices[k][0]) & (vtx_df.azimute < slices[k][1]),
        f"{str(slices[k][0])} - {str(slices[k][1])}", vtx_df.slice_label
    )

In [ ]:
result = []
for e, row in pdf.iterrows():
    for k in slices:
        _vtx = vtx_df[(vtx_df.featid == e) & (vtx_df.slice_id == k)]
        
        coords = [(row.ctr_x, row.ctr_y)]
        
        if _vtx.shape[0] < 1:
            for c in range(slices[k][0], slices[k][1] + 1):
                y = row.ctr_y + (dist * np.cos(np.radians(c)))
                x = row.ctr_x + (dist * np.sin(np.radians(c)))
                
                coords.append((x, y))
        
        _i = 0
        for i, r in _vtx.iterrows():
            if not _i:
                if r.azimute > slices[k][0]:
                    # Second point has azimute equal to minazimute
                    y = row.ctr_y + (dist * np.cos(np.radians(slices[k][0])))
                    x = row.ctr_x + (dist * np.sin(np.radians(slices[k][0])))
                    
                    coords.append((x, y))
            
            coords.append((r.new_x, r.new_y))
            
            if _i and _i + 1 == _vtx.shape[0]:
                if r.azimute < slices[k][1]:
                    y = row.ctr_y + (dist * np.cos(np.radians(slices[k][1])))
                    x = row.ctr_x + (dist * np.sin(np.radians(slices[k][1])))
                    
                    coords.append((x, y))
            
            _i += 1
        
        coords.append((row.ctr_x, row.ctr_y))
        
        poly = create_polygon(coords, api='shapely')
        
        d = {
            "geometry": poly, "featid" : e,
            "slice" : f"{str(slices[k][0])} - {str(slices[k][1])}"
        }
        
        for attr in pdf.columns.values:
            if attr == "geometry": continue
            
            d[attr] = row[attr]
        
        result.append(d)

In [ ]:
res_df = df_to_geodf(result, 'geometry', epsg)

In [ ]:
df_to_shp(res_df, outshp)